# Vector autoregression

The method of vector autoregression (VAR) is a statistical model that allows for multivariate time series, i.e., multiple time series observed at the same point in time. Therefore, it can be regarded as the more-dimensional analogue of autoregressive models. It can capture the relationship between multiple quantities as they change over time. Each variable, i.e., in our case each sales time series, has an equation modelling its evolution over time. This equation includes the variable's lagged (past) values, the lagged values of the other variables in the model, and an error term. In our case, the other variables in the model are the other time series. In our application, we want to forecast only for the next month, i.e., the horizon equals 1. We decide to only consider lags of 1, i.e., monthly lags, and therefore a VAR model of order 1.

Vector autoregression is meant to serve as a baseline for later comparison with the GTS model.

For our 57 time series with 24 observations each, we choose the train-validation-test split such that we have the first 15 observations, i.e., the ones from 09-2019 to 11-2020, in the training set, the following 3 observations, i.e., the ones from 12-2020 to 02-2021, in the validation set and the last 6 observations, i.e., the ones from 03-2021 to 08-2021, in the test set.

This implementation is based on the [implementation of vector autoregression](https://github.com/chaoshangcs/GTS/blob/main/scripts/eval_baseline_methods.py), which <span style="font-variant:small-caps;">Shang et al. (2021)</span>, the authors of the paper *Discrete Graph Structure Learning for Forecasting Multiple Time Series*, provide on GitHub together with their implementation of the GTS model.

In [1]:
import argparse
import numpy as np
import pandas as pd
from statsmodels.tsa.vector_ar.var_model import VAR
import logging
import os
import sys

## Define helper functions

In [2]:
def masked_rmse_np(preds, labels, null_val=np.nan):
    return np.sqrt(masked_mse_np(preds=preds, labels=labels, null_val=null_val))


def masked_mse_np(preds, labels, null_val=np.nan):
    with np.errstate(divide='ignore', invalid='ignore'):
        if np.isnan(null_val):
            mask = ~np.isnan(labels)
        else:
            mask = np.not_equal(labels, null_val)
        mask = mask.astype('float32')
        mask /= np.mean(mask)
        rmse = np.square(np.subtract(preds, labels)).astype('float32')
        rmse = np.nan_to_num(rmse * mask)
        return np.mean(rmse)


def masked_mae_np(preds, labels, null_val=np.nan):
    with np.errstate(divide='ignore', invalid='ignore'):
        if np.isnan(null_val):
            mask = ~np.isnan(labels)
        else:
            mask = np.not_equal(labels, null_val)
        mask = mask.astype('float32')
        mask /= np.mean(mask)
        mae = np.abs(np.subtract(preds, labels)).astype('float32')
        mae = np.nan_to_num(mae * mask)
        return np.mean(mae)


def masked_mape_np(preds, labels, null_val=np.nan):
    with np.errstate(divide='ignore', invalid='ignore'):
        if np.isnan(null_val):
            mask = ~np.isnan(labels)
        else:
            mask = np.not_equal(labels, null_val)
        mask = mask.astype('float32')
        mask /= np.mean(mask)
        mape = np.abs(np.divide(np.subtract(preds, labels).astype('float32'), labels))
        mape = np.nan_to_num(mask * mape)
        return np.mean(mape)

In [3]:
class StandardScaler:
    """
    Standard the input
    """

    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def transform(self, data):
        return (data - self.mean) / self.std

    def inverse_transform(self, data):
        return (data * self.std) + self.mean

In [4]:
def get_logger(log_dir, name, log_filename='info.log', level=logging.INFO):
    logger = logging.getLogger(name)
    logger.setLevel(level)
    if (logger.hasHandlers()): 
        logger.handlers.clear() 
    # Add file handler and stdout handler
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    file_handler = logging.FileHandler(os.path.join(log_dir, log_filename))
    file_handler.setFormatter(formatter)
    # Add console handler.
    console_formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    console_handler = logging.StreamHandler(sys.stdout)
    console_handler.setFormatter(console_formatter)
    logger.addHandler(file_handler)
    logger.addHandler(console_handler)
    # Add google cloud log handler
    logger.info('Log directory: %s', log_dir)
    return logger

In [5]:
def var_predict(df, n_forwards, n_lags, test_ratio): 
    n_sample, n_output = df.shape
    n_test = int(round(n_sample * test_ratio))
    n_train = n_sample - n_test
    df_train, df_test = df[:n_train], df[n_train:]

    scaler = StandardScaler(mean=df_train.values.mean(), std=df_train.values.std())
    data = scaler.transform(df_train.values)
    var_model = VAR(data)
    var_result = var_model.fit(n_lags)
    max_n_forwards = np.max(n_forwards)
    # Do forecasting.
    result = np.zeros(shape=(len(n_forwards), n_test, n_output))
    start = n_train - n_lags - max_n_forwards + 1
    for input_ind in range(start, n_sample - n_lags):
        prediction = var_result.forecast(scaler.transform(df.values[input_ind: input_ind + n_lags]), max_n_forwards)
        for i, n_forward in enumerate(n_forwards):
            result_ind = input_ind - n_train + n_lags + n_forward - 1
            if 0 <= result_ind < n_test:
                result[i, result_ind, :] = prediction[n_forward - 1, :]

    df_predicts = []
    for i, n_forward in enumerate(n_forwards):
        df_predict = pd.DataFrame(scaler.inverse_transform(result[i]), index=df_test.index, columns=df_test.columns)
        df_predicts.append(df_predict)
        
    val_predicts = [] 
    test_predicts = []
    for i in range(len(df_predicts)):
        val_predicts.append(df_predicts[i][0:3])
        test_predicts.append(df_predicts[i][3:])
        
    y_predict_val = val_predicts 
    y_val = df_test[0:3]
    y_predict_test = test_predicts
    y_test = df_test[3:]
        
    return y_predict_val, y_val, y_predict_test, y_test

In [6]:
def eval_var(sales, n_lags): 
    n_forwards = [1]
    y_predict_val, y_val, y_predict_test, y_test = var_predict(sales, n_forwards=n_forwards, n_lags=n_lags, test_ratio=9/24)
    logger.info('VAR (lag=%d)' % n_lags)
    logger.info('Model\tHorizon\tRMSE\tMAPE\tMAE')
    for i, horizon in enumerate(n_forwards):
        val_rmse = masked_rmse_np(preds=y_predict_val[i], labels=y_val, null_val=0)
        val_mape = masked_mape_np(preds=y_predict_val[i], labels=y_val, null_val=0)
        val_mae = masked_mae_np(preds=y_predict_val[i], labels=y_val, null_val=0)
        test_rmse = masked_rmse_np(preds=y_predict_test[i], labels=y_test, null_val=0)
        test_mape = masked_mape_np(preds=y_predict_test[i], labels=y_test, null_val=0)
        test_mae = masked_mae_np(preds=y_predict_test[i], labels=y_test, null_val=0)
        line_val = 'VAR\t%d\t%.2f\t%.2f\t%.2f' % (horizon, val_rmse, val_mape * 100, val_mae)
        line_test = 'VAR\t%d\t%.2f\t%.2f\t%.2f' % (horizon, test_rmse, test_mape * 100, test_mae)
        logger.info(line_val)
        logger.info(line_test)
    
    return y_predict_val, y_val, y_predict_test, y_test

## Forecasting

When running the GTS model with unchanged data, the error message appears that the column with index 20 (Norrbotten-Sunderbyn) is constant. To eliminate this error message and thus enable the GTS model to run, we slightly change the data in column 20 by increasing some of the entries with value 0 to value 1. We do this in the column with index 20 for the rows with index 0, 3, 8, 14 and 17. Of course, for the sake of consistency, we use the slightly modified data for both the GTS model and the three baselines.

This modification of the data has minimal impact on the forecasts and the final metrics, but allows the VAR model and the GTS model to be run.

In [7]:
logger = get_logger('data/model', 'Baseline')
sales = pd.read_hdf("data/sales.h5")
sales[sales.columns[20]][0] = 1
sales[sales.columns[20]][3] = 1 
sales[sales.columns[20]][8] = 1 
sales[sales.columns[20]][14] = 1 
sales[sales.columns[20]][17] = 1 
y_predict_val_va, y_val, y_predict_test_va, y_test =  eval_var(sales, n_lags=1)

2022-07-27 09:49:30,791 - INFO - Log directory: data/model
2022-07-27 09:49:31,445 - INFO - VAR (lag=1)
2022-07-27 09:49:31,445 - INFO - Model	Horizon	RMSE	MAPE	MAE
2022-07-27 09:49:31,463 - INFO - VAR	1	219.29	139.57	117.40
2022-07-27 09:49:31,464 - INFO - VAR	1	226.59	76.44	121.57


## Forecasts

We now take a closer look at the forecasts for both the validation set and the test set.

### Validation set

In [8]:
y_predict_val_va[0]

territory,Blekinge,Blekinge ONCO,Dalarna,Dalarna ONCO,Gävleborg-Gävle,Gävleborg-Gävle ONCO,Halland-Halmstad,Halland-Halmstad ONCO,Halland-Varberg-Falkenberg,Jämtland,...,Västra Götaland-Göteborg ONCO,Västra Götaland-Lidköping,Västra Götaland-Skövde,Västra Götaland-SÄS ONCO,Västra Götaland-Uddevalla,Örebro-Örebro,Örebro-Örebro ONCO,Östergötland-Linköping,Östergötland-Linköping ONCO,Östergötland-Norrköping
15,78.952526,351.944039,17.519475,183.869787,-49.045849,984.949982,88.787541,186.129569,58.018767,36.441764,...,2046.504474,23.558732,1.501142,1452.691910,87.673775,24.878208,452.960314,23.776681,804.322790,49.785088
16,71.723232,149.576092,21.576663,189.597717,-7.360811,1088.985620,58.766302,248.416836,26.472746,66.429691,...,1715.435348,22.094121,6.328687,1251.358259,82.256761,114.683477,341.927116,23.324039,479.735617,35.064149
17,131.632158,343.280718,18.532506,278.799965,60.921192,509.251528,4.120123,124.958339,49.900001,86.172551,...,1854.996533,2.695660,29.288638,863.996646,92.238490,111.408654,542.068514,62.420758,372.700430,41.137704


In [9]:
y_predict_val_va[0].isnull().values.any()

False

In [10]:
y_val

territory,Blekinge,Blekinge ONCO,Dalarna,Dalarna ONCO,Gävleborg-Gävle,Gävleborg-Gävle ONCO,Halland-Halmstad,Halland-Halmstad ONCO,Halland-Varberg-Falkenberg,Jämtland,...,Västra Götaland-Göteborg ONCO,Västra Götaland-Lidköping,Västra Götaland-Skövde,Västra Götaland-SÄS ONCO,Västra Götaland-Uddevalla,Örebro-Örebro,Örebro-Örebro ONCO,Östergötland-Linköping,Östergötland-Linköping ONCO,Östergötland-Norrköping
15,142.0,149.0,42.0,90.0,21.0,381.0,120.0,161.0,21.0,21.0,...,2016.0,7.0,21.0,1294.0,170.0,43.0,185.0,66.0,631.0,14.0
16,142.0,144.0,42.0,38.0,64.0,382.0,120.0,0.0,85.0,14.0,...,1397.0,0.0,35.0,1205.0,170.0,128.0,426.0,94.0,510.0,0.0
17,106.0,90.0,7.0,96.0,43.0,221.0,21.0,108.0,64.0,35.0,...,1420.0,0.0,21.0,1038.0,85.0,64.0,112.0,84.0,567.0,21.0


Other than with static forecasting and historical average forecasting, vector autoregression also creates negative sales forecasts, which is not sensible. 

There are no null values.

### Test set

In [11]:
y_predict_test_va[0]

territory,Blekinge,Blekinge ONCO,Dalarna,Dalarna ONCO,Gävleborg-Gävle,Gävleborg-Gävle ONCO,Halland-Halmstad,Halland-Halmstad ONCO,Halland-Varberg-Falkenberg,Jämtland,...,Västra Götaland-Göteborg ONCO,Västra Götaland-Lidköping,Västra Götaland-Skövde,Västra Götaland-SÄS ONCO,Västra Götaland-Uddevalla,Örebro-Örebro,Örebro-Örebro ONCO,Östergötland-Linköping,Östergötland-Linköping ONCO,Östergötland-Norrköping
18,108.815108,310.586706,13.354772,419.425784,2.390009,553.959680,-46.215195,197.409718,35.321256,60.115643,...,2063.392712,19.919132,11.904123,1062.334373,60.153444,143.142567,678.491321,10.243190,298.987284,58.490425
19,165.786464,485.137690,15.980423,68.849335,24.871768,510.924815,8.455586,168.516435,87.838630,36.794581,...,2479.092611,40.498389,50.879308,1041.217562,118.326441,164.388693,475.348878,53.977616,385.074348,15.038204
20,134.686178,379.247616,48.021356,309.568044,3.661475,532.704076,0.407515,125.834710,35.731584,44.939290,...,1915.288173,22.699636,21.158520,1315.395212,132.835531,111.564505,433.664806,42.626771,582.495296,64.200094
21,136.255665,353.916857,-8.049116,280.135353,7.606749,546.082186,-30.631653,188.426875,32.266583,44.894975,...,2411.355455,-1.716168,-5.623852,989.396583,78.605928,145.825060,647.475517,22.099270,303.566233,11.677370
22,152.760992,234.568397,25.160830,138.095542,-13.445117,677.010431,43.243002,168.607539,1.835720,7.527283,...,2312.023759,21.626212,12.883988,1287.036399,112.782409,99.676268,203.604609,36.953067,490.366664,5.513625
23,71.225205,136.440746,-18.090537,172.054972,-84.286517,1226.334203,72.371136,342.556698,-25.959306,7.389640,...,2469.156791,-20.984481,-70.225454,1578.819852,110.608358,81.438874,476.732407,-21.670807,768.887132,-39.088766


In [12]:
y_predict_test_va[0].isnull().values.any()

False

In [13]:
y_test

territory,Blekinge,Blekinge ONCO,Dalarna,Dalarna ONCO,Gävleborg-Gävle,Gävleborg-Gävle ONCO,Halland-Halmstad,Halland-Halmstad ONCO,Halland-Varberg-Falkenberg,Jämtland,...,Västra Götaland-Göteborg ONCO,Västra Götaland-Lidköping,Västra Götaland-Skövde,Västra Götaland-SÄS ONCO,Västra Götaland-Uddevalla,Örebro-Örebro,Örebro-Örebro ONCO,Östergötland-Linköping,Östergötland-Linköping ONCO,Östergötland-Norrköping
18,149.0,185.0,50.0,339.0,64.0,400.0,99.0,161.0,43.0,43.0,...,1909.0,28.0,28.0,1630.0,149.0,128.0,274.0,43.0,277.0,0.0
19,170.0,185.0,28.0,248.0,64.0,387.0,57.0,257.0,43.0,35.0,...,1796.0,21.0,35.0,903.0,213.0,149.0,200.0,35.0,603.0,0.0
20,142.0,145.0,14.0,217.0,94.0,199.0,71.0,19.0,30.0,43.0,...,1826.0,50.0,78.0,1281.0,149.0,106.0,216.0,35.0,555.0,0.0
21,248.0,248.0,64.0,138.0,85.0,181.0,123.0,232.0,21.0,64.0,...,2424.0,28.0,21.0,1351.0,201.0,106.0,515.0,78.0,689.0,0.0
22,135.0,421.0,50.0,242.0,64.0,415.0,0.0,90.0,21.0,64.0,...,3352.0,28.0,0.0,1219.0,137.0,64.0,229.0,99.0,695.0,0.0
23,152.0,282.0,71.0,235.0,43.0,193.0,57.0,216.0,103.0,71.0,...,2222.0,28.0,14.0,1686.0,191.0,57.0,121.0,101.0,769.0,0.0


Other than with static forecasting and historical average forecasting, vector autoregression also creates negative sales forecasts, which is not sensible. 

There are no null values.

## Save forecasts

Before saving the forecasts, we extend the two data frames `y_predict_val_va` and `y_predict_test_va` by the columns `time` and `type`, indicating the respective month for which the forecasts were generated and the method with which they were generated (in this case: 'vector autoregression').

In [14]:
# Create forecasts folder
route0 = "./forecasts"

if not os.path.exists(route0):
    os.mkdir(route0)

In [15]:
date_dict = dict({0: '09-2019', 1: '10-2019', 2: '11-2019', 3: '12-2019', 4: '01-2020', 5: '02-2020', 6: '03-2020', 
                  7: '04-2020', 8: '05-2020', 9: '06-2020', 10: '07-2020', 11: '08-2020', 12: '09-2020', 13: '10-2020',
                 14: '11-2020', 15: '12-2020', 16: '01-2021', 17: '02-2021', 18: '03-2021', 19: '04-2021', 20: '05-2021', 
                 21: '06-2021', 22: '07-2021', 23: '08-2021'})

### Validation set

In [16]:
y_predict_val_va[0]['time'] = y_predict_val_va[0].index.map(date_dict)
y_predict_val_va[0]['type'] = 'vector autoregression'

In [17]:
print("saving file corresponding to y_predict_val_va.pkl")
y_predict_val_va[0].to_pickle(f"{route0}/y_predict_val_va.pkl")

saving file corresponding to y_predict_val_va.pkl


In [18]:
y_predict_val_va = pd.read_pickle(f"{route0}/y_predict_val_va.pkl") 
y_predict_val_va

territory,Blekinge,Blekinge ONCO,Dalarna,Dalarna ONCO,Gävleborg-Gävle,Gävleborg-Gävle ONCO,Halland-Halmstad,Halland-Halmstad ONCO,Halland-Varberg-Falkenberg,Jämtland,...,Västra Götaland-Skövde,Västra Götaland-SÄS ONCO,Västra Götaland-Uddevalla,Örebro-Örebro,Örebro-Örebro ONCO,Östergötland-Linköping,Östergötland-Linköping ONCO,Östergötland-Norrköping,time,type
15,78.952526,351.944039,17.519475,183.869787,-49.045849,984.949982,88.787541,186.129569,58.018767,36.441764,...,1.501142,1452.691910,87.673775,24.878208,452.960314,23.776681,804.322790,49.785088,12-2020,vector autoregression
16,71.723232,149.576092,21.576663,189.597717,-7.360811,1088.985620,58.766302,248.416836,26.472746,66.429691,...,6.328687,1251.358259,82.256761,114.683477,341.927116,23.324039,479.735617,35.064149,01-2021,vector autoregression
17,131.632158,343.280718,18.532506,278.799965,60.921192,509.251528,4.120123,124.958339,49.900001,86.172551,...,29.288638,863.996646,92.238490,111.408654,542.068514,62.420758,372.700430,41.137704,02-2021,vector autoregression


### Test set

In [19]:
y_predict_test_va[0]['time'] = y_predict_test_va[0].index.map(date_dict)
y_predict_test_va[0]['type'] = 'vector autoregression'

In [20]:
print("saving file corresponding to y_predict_test_va.pkl")
y_predict_test_va[0].to_pickle(f"{route0}/y_predict_test_va.pkl")

saving file corresponding to y_predict_test_va.pkl


In [21]:
y_predict_test_va = pd.read_pickle(f"{route0}/y_predict_test_va.pkl") 
y_predict_test_va

territory,Blekinge,Blekinge ONCO,Dalarna,Dalarna ONCO,Gävleborg-Gävle,Gävleborg-Gävle ONCO,Halland-Halmstad,Halland-Halmstad ONCO,Halland-Varberg-Falkenberg,Jämtland,...,Västra Götaland-Skövde,Västra Götaland-SÄS ONCO,Västra Götaland-Uddevalla,Örebro-Örebro,Örebro-Örebro ONCO,Östergötland-Linköping,Östergötland-Linköping ONCO,Östergötland-Norrköping,time,type
18,108.815108,310.586706,13.354772,419.425784,2.390009,553.959680,-46.215195,197.409718,35.321256,60.115643,...,11.904123,1062.334373,60.153444,143.142567,678.491321,10.243190,298.987284,58.490425,03-2021,vector autoregression
19,165.786464,485.137690,15.980423,68.849335,24.871768,510.924815,8.455586,168.516435,87.838630,36.794581,...,50.879308,1041.217562,118.326441,164.388693,475.348878,53.977616,385.074348,15.038204,04-2021,vector autoregression
20,134.686178,379.247616,48.021356,309.568044,3.661475,532.704076,0.407515,125.834710,35.731584,44.939290,...,21.158520,1315.395212,132.835531,111.564505,433.664806,42.626771,582.495296,64.200094,05-2021,vector autoregression
21,136.255665,353.916857,-8.049116,280.135353,7.606749,546.082186,-30.631653,188.426875,32.266583,44.894975,...,-5.623852,989.396583,78.605928,145.825060,647.475517,22.099270,303.566233,11.677370,06-2021,vector autoregression
22,152.760992,234.568397,25.160830,138.095542,-13.445117,677.010431,43.243002,168.607539,1.835720,7.527283,...,12.883988,1287.036399,112.782409,99.676268,203.604609,36.953067,490.366664,5.513625,07-2021,vector autoregression
23,71.225205,136.440746,-18.090537,172.054972,-84.286517,1226.334203,72.371136,342.556698,-25.959306,7.389640,...,-70.225454,1578.819852,110.608358,81.438874,476.732407,-21.670807,768.887132,-39.088766,08-2021,vector autoregression
